# All D11 Calculation Pathways

This notebook provides a comprehensive analysis of snow D11 (flexural rigidity in the x-direction) calculation methods at the **slab-level** scale.

## Table of Contents

1. [Load Snow Pit Data](#1-load-snow-pit-data)
2. [Find All D11 Calculation Pathways](#2-find-all-d11-calculation-pathways)
3. [Slab-Level Analysis with ECTP Failure Layers](#3-slab-level-analysis-with-ectp-failure-layers)
   - 3.1 [Create Slabs from ECTP Failure Layers](#31-create-slabs-from-ectp-failure-layers)
   - 3.2 [Execute Pathways on ECTP Slabs](#32-execute-pathways-on-ectp-slabs)
   - 3.3 [Extract D11 Values and Success Rates](#33-extract-d11-values-and-success-rates)

## Workflow

This notebook implements a slab-level analysis approach:

### Sections 1-2: Data Preparation
1. **Load Data**: Parse all snow pit CAAML files from `examples/data/`
2. **Find Pathways**: Use graph algorithm to discover all possible D11 calculation methods

### Section 3: Slab-Level Analysis
3. **Create ECTP Slabs**: Build multi-layer slabs using ECTP (Extended Column Test with Propagation) failure layers
4. **Execute**: Run all pathways on each complete slab
5. **Extract Results**: Organize slab-level D11 calculations and analyze success rates by pathway

## Key Approach

### Slab-Level Analysis  
**Multi-layer slabs are analyzed as complete structures**:
- Slabs contain all layers above ECTP failure layers (weak layers)
- D11 is a slab-level property representing flexural rigidity
- Success requires elastic modulus calculations for **ALL layers** in a slab
- More realistic for avalanche applications where complete profiles are needed
- Reveals which methods work for entire avalanche-relevant structures

**Target Parameter**: `D11` - slab flexural rigidity in the x-direction (N⋅m)

The analysis uses the SnowPyt-MechParams execution engine with dynamic programming (caching) for efficient computation across multiple pathways.

In [1]:
# Standard library imports
import os
from pathlib import Path
from typing import List, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# Scientific computing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from uncertainties import ufloat

# SnowPyt-MechParams imports
from snowpyt_mechparams.snowpilot import parse_caaml_directory
from snowpyt_mechparams.data_structures import Pit, Layer
from snowpyt_mechparams.graph import graph
from snowpyt_mechparams.algorithm import find_parameterizations
from snowpyt_mechparams.execution import ExecutionEngine
from snowpyt_mechparams.execution.config import ExecutionConfig

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ All imports successful")

✓ All imports successful


## 1. Load Snow Pit Data

Load all CAAML XML files from `examples/data/` and create Pit objects.

In [2]:
# Path to data directory
data_dir = Path("data")

# Parse all CAAML files in the directory
print(f"Loading CAAML files from: {data_dir.absolute()}")
snow_pits_raw = parse_caaml_directory(str(data_dir))

# Create Pit objects from the parsed data
pits = [Pit.from_snow_pit(sp) for sp in snow_pits_raw]

print(f"\n✓ Successfully loaded {len(pits)} snow pits")
print(f"  Total layers across all pits: {sum(len(pit.layers) for pit in pits)}")

Loading CAAML files from: /Users/marykate/Desktop/Snow/SnowPyt-MechParams/examples/data

✓ Successfully loaded 50278 snow pits
  Total layers across all pits: 371429


## 2. Find All D11 Calculation Pathways

Use the graph and algorithm to discover all possible ways to calculate D11.

In [3]:
# Get the D11 node from the graph
d11_node = graph.get_node("D11")

# Find all parameterizations (pathways) for calculating D11
pathways = find_parameterizations(graph, d11_node)

print(f"Found {len(pathways)} pathways for calculating D11:\n")
for i, pathway in enumerate(pathways, 1):
    print(f"Pathway {i}:")
    print(pathway)
    print()

Found 32 pathways for calculating D11:

Pathway 1:
branch 1: snow_pit -- data_flow --> measured_layer_thickness -- data_flow --> zi
branch 2: snow_pit -- data_flow --> measured_density -- data_flow --> density -- data_flow --> merge_density_grain_form
branch 3: snow_pit -- data_flow --> measured_grain_form -- data_flow --> merge_density_grain_form
branch 4: snow_pit -- data_flow --> measured_grain_form -- kochle --> poissons_ratio -- data_flow --> merge_E_nu
merge branch 1: zi
merge branch 2, branch 3: merge_density_grain_form -- bergfeld --> elastic_modulus
merge branch 2, branch 3, branch 4: merge_E_nu
merge branch 1, branch 2, branch 3, branch 4: merge_zi_E_nu -- weissgraeber_rosendahl --> D11

Pathway 2:
branch 1: snow_pit -- data_flow --> measured_layer_thickness -- data_flow --> zi
branch 2: snow_pit -- data_flow --> measured_density -- data_flow --> density -- data_flow --> merge_density_grain_form
branch 3: snow_pit -- data_flow --> measured_grain_form -- data_flow --> merge_de

### Why 32 Pathways, Not More?

D11 requires three layer-level parameters, each with multiple methods:

| Parameter | Methods | Count |
|-----------|---------|-------|
| `density` | `data_flow`, `geldsetzer`, `kim_jamieson_table2`, `kim_jamieson_table5` | 4 |
| `elastic_modulus` | `bergfeld`, `kochle`, `wautier`, `schottner` | 4 |
| `poissons_ratio` | `kochle`, `srivastava` | 2 |

4 × 4 × 2 = **32 unique method combinations**.

#### The density sharing problem

`srivastava` requires `density` as an input (via the `merge_density_grain_form` node). The algorithm's backward traversal treats the density sub-path for `elastic_modulus` and the density sub-path for `srivastava` as two independent choices — internally generating a Cartesian product of 4 × 4 = 16 density combinations for each `srivastava` pathway (80 structural traversals in total).

In reality, `density` is a **single shared node** in the graph. In any given D11 pathway there is exactly one density method in use — the same method feeds both `elastic_modulus` and `srivastava`. There is no valid pathway where `geldsetzer` is used for E and `kim_jamieson_table2` is used for ν.

#### How it is resolved

`find_parameterizations` detects duplicates using `_method_fingerprint`, which converts each `Parameterization` into a canonical `parameter:method` string and discards any traversal whose fingerprint has already been seen. The 48 redundant structural paths are removed before the function returns, leaving exactly the 32 unique combinations shown in the cell above.

## 3. Slab-Level Analysis with ECTP Failure Layers

Analyze D11 calculation pathways at the slab level, using actual slabs created from ECTP (Extended Column Test with Propagation) failure layers.

**Note**: D11 is a slab-level property that requires elastic modulus calculations for all layers in the slab.

### 3.1 Create Slabs from ECTP Failure Layers

In [4]:
# Create slabs from pits using ECTP failure layers
# Each slab contains all layers above a weak layer identified by an ECTP test
ectp_slabs = []

print("Creating slabs from ECTP failure layers...")
pits_with_ectp = 0
pits_without_ectp = 0

for pit in pits:
    # Create slabs using ECTP_failure_layer definition
    slabs = pit.create_slabs(weak_layer_def="ECTP_failure_layer")
    
    if slabs:
        pits_with_ectp += 1
        for slab in slabs:
            ectp_slabs.append({
                'slab': slab,
                'pit_id': pit.pit_id,
                'slab_id': slab.slab_id,
                'n_layers': len(slab.layers),
                'weak_layer_depth': slab.weak_layer.depth_top if slab.weak_layer else None
            })
    else:
        pits_without_ectp += 1

print(f"\n✓ Slab creation complete!")
print(f"  Pits with ECTP tests:     {pits_with_ectp}")
print(f"  Pits without ECTP tests:  {pits_without_ectp}")
print(f"  Total slabs created:      {len(ectp_slabs)}")

Creating slabs from ECTP failure layers...

✓ Slab creation complete!
  Pits with ECTP tests:     12347
  Pits without ECTP tests:  37931
  Total slabs created:      14776


### 3.2 Execute Pathways on ECTP Slabs

`find_parameterizations` has already removed duplicate structural traversals, so the execution engine receives exactly **32 unique method combinations** and runs each one exactly once per slab.

In [5]:
# Initialize the execution engine
engine = ExecutionEngine(graph)

# Exclude method uncertainty so that reported uncertainty reflects only
# propagated input measurement uncertainty, not the regression standard error.
config = ExecutionConfig(include_method_uncertainty=False)

# Execute all pathways on each ECTP slab
slab_results: Dict[str, Any] = {}

print("Executing all D11 pathways on ECTP slabs...")
print(f"This will execute {len(pathways)} pathways on {len(ectp_slabs)} slabs\n")

successful_slabs = 0
failed_slabs = 0

for slab_idx, slab_info in enumerate(ectp_slabs, 1):
    
    slab = slab_info['slab']
    slab_id = slab_info['slab_id']
    
    try:
        # Execute ALL pathways on this slab
        results = engine.execute_all(slab, "D11", config=config)
        
        # Store results with slab metadata
        slab_results[slab_id] = {
            'execution_results': results,
            'pit_id': slab_info['pit_id'],
            'n_layers': slab_info['n_layers'],
            'weak_layer_depth': slab_info['weak_layer_depth']
        }
        successful_slabs += 1
            
    except Exception as e:
        # Skip slabs that cause errors
        failed_slabs += 1
        pass

print(f"\n✓ Execution complete!")
print(f"  Successful: {successful_slabs} slabs ({100*successful_slabs/len(ectp_slabs):.1f}%)")
print(f"  Failed:     {failed_slabs} slabs ({100*failed_slabs/len(ectp_slabs):.1f}%)")

Executing all D11 pathways on ECTP slabs...
This will execute 32 pathways on 14776 slabs


✓ Execution complete!
  Successful: 14776 slabs (100.0%)
  Failed:     0 slabs (0.0%)


### 3.3 Extract D11 Values and Success Rates

Extract slab-level D11 calculations and analyze success rates by pathway.

In [6]:
# Extract D11 values and count successful slabs per pathway
# A slab is considered successful for a pathway if D11 is calculated successfully

d11_data = []
pathway_slab_success = {}

for slab_id, result_info in slab_results.items():
    execution_results = result_info['execution_results']
    pit_id = result_info['pit_id']
    n_layers_in_slab = result_info['n_layers']
    
    # Check each pathway for this slab
    for pathway_desc, pathway_result in execution_results.pathways.items():
        # Get the full pathway (density → elastic_modulus → poissons_ratio)
        density_method = pathway_result.methods_used.get('density', 'unknown')
        e_mod_method = pathway_result.methods_used.get('elastic_modulus', 'unknown')
        pr_method = pathway_result.methods_used.get('poissons_ratio', 'unknown')
        full_pathway = f"{density_method} → {e_mod_method} → {pr_method}"
        
        # Initialize pathway tracking if not exists
        if full_pathway not in pathway_slab_success:
            pathway_slab_success[full_pathway] = {
                'successful_slabs': 0,
                'failed_slabs': 0,
                'slab_ids': [],
                'density_method': density_method,
                'e_mod_method': e_mod_method,
                'pr_method': pr_method,
                'd11_values': []
            }
        
        # Check if D11 was calculated successfully for this slab
        d11_calculated = False
        d11_value = None
        
        for trace in pathway_result.computation_trace:
            if trace.parameter == "D11" and trace.success and trace.output is not None:
                d11_calculated = True
                
                # Extract D11 value and uncertainty
                if hasattr(trace.output, 'nominal_value'):
                    # ufloat from uncertainties package
                    d11_value = trace.output.nominal_value
                    d11_std = trace.output.std_dev
                else:
                    # Handle various output types
                    try:
                        if isinstance(trace.output, (list, tuple)):
                            d11_value = float(trace.output[0]) if len(trace.output) > 0 else None
                        else:
                            d11_value = float(trace.output)
                        d11_std = 0.0
                    except (TypeError, ValueError, IndexError):
                        d11_value = None
                
                if d11_value is not None:
                    d11_data.append({
                        'slab_id': slab_id,
                        'pathway_description': pathway_desc,
                        'full_pathway': full_pathway,
                        'density_method': density_method,
                        'e_mod_method': e_mod_method,
                        'pr_method': pr_method,
                        'pit_id': pit_id,
                        'n_layers': n_layers_in_slab,
                        'D11': d11_value,
                        'D11_std': d11_std,
                        'cached': trace.cached
                    })
                    pathway_slab_success[full_pathway]['d11_values'].append(d11_value)
                break
        
        # Track success/failure for this slab and pathway
        if d11_calculated and d11_value is not None:
            pathway_slab_success[full_pathway]['successful_slabs'] += 1
            pathway_slab_success[full_pathway]['slab_ids'].append(slab_id)
        else:
            pathway_slab_success[full_pathway]['failed_slabs'] += 1

# Create DataFrame of D11 values and compute per-slab relative uncertainty
df_d11 = pd.DataFrame(d11_data)

if len(df_d11) > 0:
    df_d11['d11_relative_uncertainty'] = np.where(
        df_d11['D11'] != 0,
        df_d11['D11_std'] / df_d11['D11'],
        np.nan
    )

print(f"Extracted {len(df_d11)} D11 calculations")
if len(df_d11) > 0:
    print(f"  Across {df_d11['pathway_description'].nunique()} unique pathways")
    print(f"  From {df_d11['slab_id'].nunique()} unique slabs")
    print(f"  Spanning {df_d11['pit_id'].nunique()} pits")

print("\n" + "=" * 80)
print("SLAB-LEVEL D11 ANALYSIS SUMMARY (ECTP Failure Layers)")
print("=" * 80)

print(f"\n📊 Slab Dataset Overview:")
print(f"   • Total slabs created:             {len(ectp_slabs)}")
print(f"   • Slabs successfully processed:    {successful_slabs}")
print(f"   • Total layers in all slabs:       {sum(s['n_layers'] for s in ectp_slabs)}")

print(f"\n🔬 Success Rate by Full Pathway (Density → Elastic Modulus → Poisson's Ratio):")
print(f"\n  {'Full Pathway':<55s} {'Slabs':>14s} {'Coverage':>9s} {'Avg Rel. Uncertainty':>22s}")
print(f"  {'-'*102}")

# Sort pathways by success count (descending)
sorted_pathways = sorted(
    pathway_slab_success.items(),
    key=lambda x: x[1]['successful_slabs'],
    reverse=True
)

total_slabs = len(ectp_slabs)
for pathway, stats in sorted_pathways:
    total = stats['successful_slabs'] + stats['failed_slabs']
    pct = 100 * stats['successful_slabs'] / total_slabs if total_slabs > 0 else 0

    # Average relative uncertainty across successful slabs for this pathway
    if len(df_d11) > 0 and pathway in df_d11['full_pathway'].values:
        mask = df_d11['full_pathway'] == pathway
        avg_rel_unc = df_d11[mask]['d11_relative_uncertainty'].mean()
        unc_str = f"{avg_rel_unc:>18.1%}"
    else:
        unc_str = f"{'N/A':>18s}"

    print(f"  - {pathway:<53s} {stats['successful_slabs']:>6d} / {total_slabs:<6d} ({pct:>5.1f}%)    {unc_str}")

print()
print("  Note: Uncertainty reflects propagated input measurement uncertainties only")
print("  (method regression standard error excluded). D11 is a slab-level integral")
print("  over all layers; its relative uncertainty accumulates contributions from")
print("  layer thickness (±5%), density (via HHI ±0.67 or direct ±10%), and grain")
print("  size (±0.5 mm) propagated through the full elastic modulus and plate-theory")
print("  calculation chain.")
print(f"\n{'='*80}")
print(f"Total unique pathways: {len(pathway_slab_success)}")
print(f"\nNote: Success requires D11 calculation for the entire slab,")
print(f"      which depends on elastic modulus AND Poisson's ratio calculations for ALL layers.")
print("=" * 80)

Extracted 8495 D11 calculations
  Across 32 unique pathways
  From 827 unique slabs
  Spanning 721 pits

SLAB-LEVEL D11 ANALYSIS SUMMARY (ECTP Failure Layers)

📊 Slab Dataset Overview:
   • Total slabs created:             14776
   • Slabs successfully processed:    14776
   • Total layers in all slabs:       58126

🔬 Success Rate by Full Pathway (Density → Elastic Modulus → Poisson's Ratio):

  Full Pathway                                                     Slabs  Coverage   Avg Rel. Uncertainty
  ------------------------------------------------------------------------------------------------------
  - geldsetzer → wautier → kochle                            745 / 14776  (  5.0%)                 41.7%
  - kim_jamieson_table2 → wautier → kochle                   745 / 14776  (  5.0%)                 40.8%
  - geldsetzer → schottner → kochle                          745 / 14776  (  5.0%)                 74.8%
  - kim_jamieson_table2 → schottner → kochle                 745 / 14776  (  